In [1]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
import csv
import datetime
import time
import os
from selenium import webdriver

import matplotlib.pyplot as plt
import collections
from openpyxl import load_workbook

import math
import statistics
import numpy as np



In [2]:
def set_driver(url):

    ser = Service(DRIVER_PATH)
    options = webdriver.ChromeOptions()  # Set up Chinese
    # options.add_argument('--headless')
    options.add_argument('lang=zh_CN.UTF-8')  # Replacement of head
    options.add_argument(
        'User-Agent="Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36"')
    options.add_argument("--disable-infobars")
    options.add_argument("--headless")
    driver = webdriver.Chrome(
        service=ser,
        options=options)

    driver.get(url)
    print("finish setup driver")
    return driver


def scroll_to_button():
    reached_page_end = False
    last_height = driver.execute_script("return document.body.scrollHeight")

    while not reached_page_end:
        driver.find_element_by_xpath('//body').send_keys(Keys.END)
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if last_height == new_height:
            reached_page_end = True
        else:
            last_height = new_height
    print("reach end")
    # driver.quit()
    
def init_filename(source, subject):
    # /呱吉/PTT.csv
    # /呱吉/FB.csv
    # ...
    folder = f"/{source}/"
    return subject + folder


def write_file():
    out_filename = ""
    with open(out_filename, 'a') as out_file:
        for line in in_file:

            out_file.write(parsed_line)


def init_csv(file_name):

    with open(file_name, 'w') as out_file:
        write = csv.writer(out_file)
        header = ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']
        # PTT, 呱吉吃大便, http://, '2021-11-17', XXXX, XXXXX, 23
        write.writerow(header)


def string_to_datetime(s):
    return datetime.datetime.strptime(s, '%Y/%m/%d')


In [3]:
DRIVER_PATH = "/Users/bobo/OneDrive - 台灣微軟/0. bobo/Codes/stratechery crawler/chromedriver"
keywords_url = "https://www.ettoday.net/news_search/doSearch.php?keywords=館長"
tag_url = "https://www.ettoday.net/news/tag/統神/"


In [5]:
# tag search
# driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
driver = set_driver(tag_url)

reached_page_end = False
last_height = driver.execute_script("return document.body.scrollHeight")
time.sleep(5)
while not reached_page_end:
    body = driver.find_element_by_xpath('//body')
    print(body)
    body.send_keys(Keys.END)
    time.sleep(3)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if last_height == new_height:
        reached_page_end = True
    else:
        last_height = new_height
print("reach end")

time.sleep(10)

block_div = driver.find_element_by_class_name("c1")
title_list = block_div.find_elements_by_xpath(
    ".//div[@class='piece clearfix']")

press = title_list[-1].find_element_by_tag_name(
    "h3").find_element_by_tag_name("a")
press.send_keys(Keys.NULL)

print("len: ", len(title_list))

filename = "test.csv"
with open(filename, 'a') as f:
    write = csv.writer(f)

    header = ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']
    dic = {i: "" for i in header}

    for i in title_list:
        dic['Source'] = "ettoday"
        dic['Title'] = i.find_element_by_tag_name('h3').text.replace("  ", " ")
        dic['Link'] = i.find_element_by_tag_name('a').get_attribute("href")
        dic['Date'] = i.find_element_by_class_name('date').text
        dic['Summary'] = i.find_element_by_class_name('summary').text
        dic['Text'] = ""
        dic['Like'] = 0

        line = dic.values()
        write.writerow(line)


<ipython-input-2-908dfb727e1e>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(
<ipython-input-5-1b9b660b975d>:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  body = driver.find_element_by_xpath('//body')


<selenium.webdriver.remote.webelement.WebElement (session="04b4f270126ccdd0ee3e4831ad1a2991", element="84be087d-24c1-41a5-ae6a-914ab7c47f43")>
reach end


<ipython-input-5-1b9b660b975d>:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  block_div = driver.find_element_by_class_name("c1")
/Users/bobo/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:425: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
/Users/bobo/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:340: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
/Users/bobo/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:445: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are dep

len:  20


In [121]:
# search by keywords
driver = set_driver(keywords_url)


<ipython-input-120-908dfb727e1e>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


In [122]:
def crawl_page():
    block_div = driver.find_element(By.ID, "primary")
    box_list = block_div.find_elements(By.CLASS_NAME, "box_2")


    filename = "test.csv"
    with open(filename, 'a') as f:
        write = csv.writer(f)

        header = ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']
        dic = {i: "" for i in header}

        for box in box_list:
            dic['Source'] = "ettoday"
            dic['Title'] = box.find_element(By.TAG_NAME, "h2").text.replace("\u3000", " ")
            dic['Link'] = box.find_element(By.TAG_NAME, 'a').get_attribute("href")
            dic['Date'] = box.find_element(By.CLASS_NAME, 'date').text.split("/ ")[1].replace(")", "")
            dic['Summary'] = box.find_element(By.CLASS_NAME, 'detail').text
            dic['Text'] = ""
            dic['Like'] = 0

            line = dic.values()
            write.writerow(line)


def check_exists_by_class(class_name):
    try:
        driver.find_element(By.CLASS_NAME, class_name)
    except NoSuchElementException:
        return False
    return True

def get_max_page():
    info = driver.find_element(By.CLASS_NAME, "info").text
    for t in ["第", "頁", "共", "頁"]:
        info = info.replace(t, "")
    info = info.replace(" ", "")
    current_page, max_page = map(int, info.split("|"))
    current_page, max_page
    return max_page


In [ ]:
for i in range(get_max_page()):
    crawl_page()
    time.sleep(5)
    next_page = driver.find_element_by_xpath("//*[text()=' > ']")
    next_page.click()
